# Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('always')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
#!pip install pandas
#!pip install --upgrade langchain langgraph langchain_openapi

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# [DEPRECATED]
from langchain.chains import LLMChain

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [22]:
# [DEPRECATED]
#chain = LLMChain(llm=llm, prompt=prompt)

# It produces a Runnable sequence, not a LLMChain
chain = prompt | llm

In [23]:
product = "Queen Size Sheet Set"
chain.invoke(product)

AIMessage(content='Regal Linens', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 24, 'total_tokens': 28, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a7f12f9a-0cdc-4c3d-827f-c09da700411d-0', usage_metadata={'input_tokens': 24, 'output_tokens': 4, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

## SimpleSequentialChain

In [24]:
from langchain.chains import SimpleSequentialChain
from langchain.chains.sequential import SimpleSequentialChain

In [25]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
# SimpleSequentialChain does not support (first_prompt | llm)
chain_one = LLMChain(llm=llm, prompt=first_prompt)
#chain_one = first_prompt | llm

In [26]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
# SimpleSequentialChain does not support (second_prompt | llm)
chain_two = LLMChain(llm=llm, prompt=second_prompt)
#chain_two = second_prompt | llm

In [27]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

In [18]:
# Run the overall chain
output = overall_simple_chain.invoke({"input": product})
print(output)



> Entering new SimpleSequentialChain chain...
"Royal Linens Co."
"Royal Linens Co. offers luxurious bedding and linens fit for royalty, with a wide selection of high-quality products for every customer."

> Finished chain.
{'input': 'Queen Size Sheet Set', 'output': '"Royal Linens Co. offers luxurious bedding and linens fit for royalty, with a wide selection of high-quality products for every customer."'}


## SequentialChain

In [81]:
from langchain.chains.sequential import SequentialChain


In [141]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

# chain 1: input= Review and output= English_Review
# SequentialChain does not support (prompt | llm)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")
#chain_one = first_prompt | llm

In [142]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

# chain 2: input= English_Review and output= summary
# SequentialChain does not support (prompt | llm)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")
#chain_two = second_prompt | llm

In [143]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)

# chain 3: input= Review and output= language
# SequentialChain does not support (prompt | llm)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")
#chain_three = third_prompt | llm


In [144]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

# chain 4: input= summary, language and output= followup_message
# SequentialChain does not support (prompt | llm)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")
#chain_four = fourth_prompt | llm


In [145]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [146]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed in the taste and quality of the product, suspecting it may be an old batch or counterfeit.',
 'followup_message': "Cher client, nous vous remercions de votre retour d'expérience. Nous sommes désolés d'apprendre que vous n'avez pas été satisfait de la saveur et de la qualité de notre produit. Nous prenons votre préoccupation très au sérieux et nous vous assurons que nous allons immédiatement enquêter pour déterminer s'il s'agit d'un lot périmé ou contrefait. Votre satisfaction est notre priorité et nous ferons tout notre possible pour rectifier cette situation. Nous vous tiendrons informé des résult

### New way to combine chains ...

Sequential Chain is the old Langchain Style of combining Chains. You might want to combine chains using the following LCEL syntax:

    combined_chain = chain1 | chain2

In [169]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


output_parser = StrOutputParser()

# Now it supports | syntax
chain_one   = first_prompt  | llm | {"English_Review": output_parser}
chain_two   = second_prompt | llm | {"summary": output_parser}
chain_three = third_prompt  | llm | {"language": output_parser}
chain_four  = fourth_prompt | llm | {"followup_message": output_parser}



In [192]:
raw_review = df.Review[5]

review = {"Review": raw_review}

In [171]:
english_review = chain_one.invoke(review)
print(english_review)

{'English_Review': "I find the taste mediocre. The foam doesn't last, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"}


In [172]:
summary = chain_two.invoke(english_review)
print(summary)

{'summary': 'The reviewer is disappointed with the taste of the product, questioning if it is an old batch or counterfeit.'}


In [173]:
language = chain_three.invoke(review)
print(language)

{'language': 'French'}


In [184]:
summary_and_language = summary
summary_and_language['language'] = language['language']

followup_message = chain_four.invoke(summary_and_language)
print(followup_message)

{'followup_message': "Cher client, nous sommes désolés d'apprendre que vous n'êtes pas satisfait du goût de notre produit. Nous vous assurons que tous nos produits sont frais et authentiques. Nous vous encourageons à nous contacter directement pour résoudre ce problème et nous assurerons que vous receviez un produit de qualité. Merci pour votre retour d'information."}


### How to combine chains with pipes?

In [223]:
model_parser = llm | StrOutputParser()

english_review = first_prompt | {"English_Review": model_parser, "Review": RunnablePassthrough()}

summary = second_prompt | model_parser

language = third_prompt | {"language": model_parser}

followup_message = fourth_prompt | model_parser

assistant = english_review | {"summary":summary, "language":language} | followup_message

In [224]:
assistant.invoke(review)

"Merci pour votre avis honnête sur le produit. Il est regrettable d'apprendre que vous avez trouvé le goût médiocre et que la mousse ne tient pas bien. Il est possible que le produit que vous avez reçu soit une ancienne série ou une contrefaçon, ce qui expliquerait la différence de goût par rapport à ceux achetés en magasin. Nous vous recommandons de contacter le vendeur pour clarifier cette situation. Nous espérons que vous pourrez bientôt profiter pleinement du produit comme vous le faisiez auparavant en magasin."

Improving by using: 
- RunnablePassthrough
- RunnableLambda
- RunnableParallel

In [19]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough

# load .env file
from dotenv import load_dotenv
_ = load_dotenv()

# get the review in french
import pandas as pd
df = pd.read_csv('Data.csv')
review = df.Review[5]

# Input.: review (original)
# Output: english_review
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{review}"
)

# Input.: english_review
# Output: summary
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in 1 sentence:"
    "\n\n{english_review}"
)

# Input.: review (original)
# Output: language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:"
    "\n\n{review}"
)

# Input.: summary and language
# Output: followup_message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)

def showPrompt(prompt):
    print(">", prompt.messages[0].content, "\n")
    return prompt

def showResult(ai_msg):
    print("A:", ai_msg, "\n")
    return ai_msg

model = showPrompt | ChatOpenAI() | StrOutputParser() | showResult

chain = RunnableParallel({"english_review": first_prompt | model, "language": third_prompt | model}) \
        | {"summary": second_prompt | model, "language": RunnableLambda(lambda x: x["language"])} \
        | {"followup_message": fourth_prompt | model}

chain.invoke({"review": review})

> Translate the following review to english:

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !? 

> What language is the following review:

Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
Vieux lot ou contrefaçon !? 

A: French 

A: I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!? 

> Summarize the following review in 1 sentence:

I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!? 

A: The reviewer is disappointed with the taste and quality of the product, suspecting that it may be an old batch or counterfeit. 

> Write a follow up response to the following summary in the specified langu

{'followup_message': "Cher client,\n\nNous sommes désolés d'apprendre que vous n'êtes pas satisfait de la qualité de notre produit. Nous vous assurons que nous mettons tout en œuvre pour garantir la fraîcheur et l'authenticité de nos produits. N'hésitez pas à nous contacter directement pour que nous puissions enquêter sur cette situation et résoudre tout problème éventuel. Votre satisfaction est notre priorité.\n\nCordialement, \n\nL'équipe de [Votre entreprise]"}

In [ ]:
# > Translate the following review to english:
# 
# Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
# Vieux lot ou contrefaçon !? 
# 
# > What language is the following review:
# 
# Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...
# Vieux lot ou contrefaçon !? 
# 
# A: French 
# 
# A: I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!? 
# 
# > Summarize the following review in 1 sentence:
# 
# I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!? 
# 
# A: The reviewer is disappointed with the taste and quality of the product, suspecting that it may be an old batch or counterfeit. 
# 
# > Write a follow up response to the following summary in the specified language:
# 
# Summary: The reviewer is disappointed with the taste and quality of the product, suspecting that it may be an old batch or counterfeit.
# 
# Language: French 
# 
# A: Cher client,
# 
# Nous sommes désolés d'apprendre que vous n'êtes pas satisfait de la qualité de notre produit. Nous vous assurons que nous mettons tout en œuvre pour garantir la fraîcheur et l'authenticité de nos produits. N'hésitez pas à nous contacter directement pour que nous puissions enquêter sur cette situation et résoudre tout problème éventuel. Votre satisfaction est notre priorité.
# 
# Cordialement, 
# 
# L'équipe de [Votre entreprise] 

Improving by using
- LangGraph

In [ ]:
# load .env file
from dotenv import load_dotenv
_ = load_dotenv()

# get the review in french
import pandas as pd
df = pd.read_csv('Data.csv')
review = df.Review[5]

from typing import TypedDict, Annotated

from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

prompts = [
    "What language is the following review:\n\n{review}",
    "Translate the following review to english:\n\n{review}",    
    "Summarize the following review in 1 sentence:\n\n{english_review}",
    "Write a follow up response to the following summary in the specified language:\n\nSummary: {summary}\n\nLanguage: {language}"
]

def showPrompt(prompt):
    print(">", prompt.messages[0].content, "\n")
    return prompt

def showResult(ai_msg):
    print("A:", ai_msg, "\n")
    return ai_msg

model = showPrompt | ChatOpenAI() | StrOutputParser() | showResult

chain = RunnableParallel({"english_review": first_prompt | model, "language": third_prompt | model}) \
        | {"summary": second_prompt | model, "language": RunnableLambda(lambda x: x["language"])} \
        | {"followup_message": fourth_prompt | model}

chain.invoke({"review": review})

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [ ]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

Reminder: Download your notebook to you local computer to save your work.